**Import Packages**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

**Prepare Data**

*Dowwnload Data*

In [2]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

*Normalize Data*

In [3]:
X_train = X_train/255
X_test = X_test/255

**Create Autoencoders**

*Encoder Model*

In [4]:
model = Sequential()
model.add(Flatten(input_shape=[28,28]))
model.add(Dense(units=392,activation='relu'))

*Decoder Model*

In [5]:
decoder = Sequential()
decoder.add(Dense(units=5000,input_shape=[392],activation='relu'))
decoder.add(Dense(units=784,activation='sigmoid'))
decoder.add(Reshape([28,28]))

*Autoencoder Model*

In [6]:
autoencoder = Sequential([model,decoder])

*Compile Autoencoder*

In [7]:
#autoencoder.compile(loss='binary_crossentropy',optimizer=SGD(lr=1.5),metrics=['accuracy'])
#autoencoder.compile(loss='mean_squared_error',optimizer=SGD(lr=1.5),metrics=['mean_squared_error'])
autoencoder.compile(loss='mean_squared_error',optimizer=Adam(lr=0.00001),metrics=['mean_squared_error'])

*Train Autoencoder*

In [ ]:
#autoencoder.fit(X_train,X_train,epochs=5,validation_data=(X_test,X_test))
#autoencoder.fit(X_train,X_train,epochs=100,validation_data=(X_test,X_test))
autoencoder.fit(X_train,X_train,epochs=1500,validation_data=(X_test,X_test))

*Save Model*

In [ ]:
autoencoder.save('alr')

*Load Model*

In [8]:
autoencoder = tf.keras.models.load_model('alr')

**Lossless Recovery**

**1 Simple Compensation**

*1-1 float*

In [9]:
#
x0 = X_test[:1]*1
y0 = autoencoder.predict(x0[:1])
xu0 = tf.cast(x0*255, tf.uint8)
yu0 = tf.cast(y0*255, tf.uint8)

#
print(0,tf.image.psnr(yu0,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(yu0, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

#
lmd = 1

#
x = y0*1

#
for ii in range(1,1+1):

    # postive
    for k in range(1,1+1):
        y = autoencoder.predict(x[:1])
        x = x+(y0-y)*lmd
        for i in range(0,28,1):
            for j in range(0,28,1):
                sum = x[0][i][j]
                if (sum<0):
                    x[0][i][j] = 0
                elif (sum>1):
                    x[0][i][j] = 1
     
        xu = tf.cast(x*255, tf.uint8)      
        print(k,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

    # negative
    for k in range(1,0+1):
        y = autoencoder.predict(x[:1])
        x = x-(y0-y)*lmd
        for i in range(0,28,1):
            for j in range(0,28,1):
                sum = x[0][i][j]
                if (sum<0):
                    x[0][i][j] = 0
                elif (sum>1):
                    x[0][i][j] = 1
     
        xu = tf.cast(x*255, tf.uint8)      
        print(k,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())
    

0 35.10557 0.9811089
1 40.164032 0.98720944


*1-2 integer*

In [10]:
#
x0 = X_test[:1]*1
xu0 = tf.cast(x0*255, tf.uint8)
xi0 = tf.cast(xu0, tf.int16)

#
y0 = autoencoder.predict(x0[:1])
yu0 = tf.cast(y0*255, tf.uint8)
yi0 = tf.cast(yu0, tf.int16)

#
print(0,tf.image.psnr(yu0,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(yu0, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

#
lmd = 1

#
x = y0*1
xu = tf.cast(x*255, tf.uint8)
xi = tf.cast(xu, tf.int16)

#
for ii in range(1,1+1):

    # positive
    for k in range(1,1+1):
        #print(k)
        y = autoencoder.predict(x[:1])
        yu = tf.cast(y*255, tf.uint8)
        yi = tf.cast(yu, tf.int16)
        xi = xi+(yi0-yi)*lmd
        xn = xi.numpy()
        for i in range(0,28,1):
            for j in range(0,28,1):
                sum = xn[0][i][j]
                if (sum<0):
                    xn[0][i][j] = 0
                elif (sum>255):
                    xn[0][i][j] = 255
        xi = tf.convert_to_tensor(xn)
        x = xi/255
        xu = tf.cast(x*255, tf.uint8)      
        xi = tf.cast(xu, tf.int16)
        print(k,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

    # negative
    for k in range(1,0+1):
        #print(k)
        y = autoencoder.predict(x[:1])
        yu = tf.cast(y*255, tf.uint8)
        yi = tf.cast(yu, tf.int16)
        xi = xi-(yi0-yi)*lmd
        xn = xi.numpy()
        for i in range(0,28,1):
            for j in range(0,28,1):
                sum = xn[0][i][j]
                if (sum<0):
                    xn[0][i][j] = 0
                elif (sum>255):
                    xn[0][i][j] = 255
        xi = tf.convert_to_tensor(xn)
        x = xi/255
        xu = tf.cast(x*255, tf.uint8)      
        xi = tf.cast(xu, tf.int16)
        print(k,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())
    

0 35.10557 0.9811089
1 40.093826 0.9853961


**2 1x1 Searching**

*2-1 Step 1*

In [11]:
# Intialization
x0 = X_test[:1]*1
y0 = autoencoder.predict(x0[:1])
xu0 = tf.cast(x0*255, tf.uint8).numpy()
yu0 = tf.cast(y0*255, tf.uint8).numpy()

#
x = y0*1
y = autoencoder.predict(x[:1])
ex = x0-x
ey = y0-y
xu = tf.cast(x*255, tf.uint8)
yu = tf.cast(y*255, tf.uint8)
exu = tf.cast(ex*255, tf.int16).numpy()
eyu = tf.cast(ey*255, tf.int16).numpy()
xi = tf.cast(xu, tf.int16)

print(0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

file = open("result_alr_1x1_searching_step_1.txt","a")
file.write(str([0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
file.write('\n')

#
for ii in range(1,20+1,1):
    for i in range(0,28,1):
        for j in range(0,28,1):
            """
            nb = 1
            for m in range(-1,+1):
                for n in range(-1,+1):
                    if (((i+m)>=0) and ((i+m)<28) and ((j+n)>=0) and ((j+n)<28) and (eyu[0][i+m][j+n]!=0)):
                        nb = nb+1
            if (nb>=1):
            """
            if (eyu[0][i][j]!=0):
                k0 = tf.cast(x[0][i][j]*255,tf.int16)
                #for k in range(k0-1,k0+1+1,1):
                for k in range(k0-ii,k0+ii+1,1):
                #for k in range(k0-ii,k0+ii+1,ii):
                    if ((k==k0) or (k<0) or (k>255)):
                        continue
                    #x1 = x*1
                    #x1[0][i][j] = k/255
                    xi1 = xi*1
                    xn1 = xi1.numpy()
                    xn1[0][i][j] = k
                    xi1 = tf.convert_to_tensor(xn1)
                    x1 = xi1/255
                    y1 = autoencoder.predict(x1[:1])
                    ex1 = x0-x1
                    ey1 = y0-y1
                    exu1 = tf.cast(ex1*255, tf.int16).numpy()
                    eyu1 = tf.cast(ey1*255, tf.int16).numpy()
                    """
                    cnt1 = 0
                    for m in range(0,28,1):
                        for n in range(0,28,1):
                            #if ((ey1[0][i][j]==ey[0][i][j]) or ((ey1[0][i][j]*ey[0][i][j]>0) and (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])))):
                            #if ((tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])) or (((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))))):
                            if (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])):
                            #if ((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))):                            
                                cnt1 = cnt1 +1
                    if (cnt1==(28*28)):
                    #if ((cnt1==(28*28)) and (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey)))):
                    """
                    #if ((k>k0) and (tf.reduce_mean(ey1)<0)):
                    #if ((((k<k0) and (tf.reduce_mean(ey1)>0)) or ((k>k0) and (tf.reduce_mean(ey1)<0))) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey)))):
                    #if (((k<k0) and (tf.reduce_mean(ey1)<tf.reduce_mean(ey))) or ((k>k0) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))))):
                    #if (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))):
                    #if ((tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))) and (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu)))):
                    if (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))):
                    #if (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu))):
                        #x[0][i][j] = x1[0][i][j]
                        xi = xi1*1
                        x = xi/255
                        ey = ey1*1
                        eyu = eyu1*1
                        
                        xu = tf.cast(x*255, tf.uint8)       
                        print(ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())
                        file.write(str([ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
                        file.write('\n')
                
file.close()
                
                

0 0 0 0 35.10557 0.9811089
2 7 6 71 84 35.121056 0.9811089
2 7 7 197 185 35.135483 0.9811089
2 7 8 175 159 35.154423 0.981109
2 7 9 157 151 35.162247 0.9811091
2 7 10 49 60 35.168682 0.9811089
2 7 10 50 60 35.17457 0.98110896
2 7 11 26 36 35.18046 0.981109
2 7 11 27 36 35.185802 0.98110896
2 7 14 9 0 35.197063 0.9811092
2 7 15 8 0 35.207222 0.9811094
2 7 16 7 0 35.216274 0.98110944
2 7 17 6 0 35.224213 0.98110944
2 7 18 1 0 35.226482 0.98110944
2 7 19 5 0 35.2333 0.98110944
2 7 20 7 0 35.2424 0.98110944
2 8 5 4 0 35.248104 0.98110944
2 8 6 214 222 35.252956 0.98110944
2 8 6 215 222 35.257244 0.98110944
2 8 7 246 254 35.262104 0.98110944
2 8 7 247 254 35.2664 0.98110944
2 8 8 240 254 35.27471 0.9811093
2 8 8 241 254 35.28247 0.98110944
2 8 9 243 254 35.289085 0.98110944
2 8 9 244 254 35.295135 0.9811095
2 8 10 243 254 35.301773 0.9811094
2 8 10 244 254 35.307842 0.98110944
2 8 12 216 198 35.329876 0.9811121
2 8 13 204 198 35.338024 0.9811147
2 8 14 216 198 35.36021 0.9811188
2 8 15 217 

*2-2 Step 2*

In [12]:
print(0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

file = open("result_alr_1x1_searching_step_2.txt","a")
file.write(str([0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
file.write('\n')

#
#th = 0.006*tf.reduce_max(tf.abs(ey))
th =0.05/255

#
for ii in range(1,20+1,1):
    for i in range(0,28,1):
        for j in range(0,28,1):
            """
            nb = 1
            for m in range(-1,+1):
                for n in range(-1,+1):
                    if (((i+m)>=0) and ((i+m)<28) and ((j+n)>=0) and ((j+n)<28) and (eyu[0][i+m][j+n]!=0)):
                        nb = nb+1
            if (nb>=1):
            """
            if (tf.abs(ey[0][i][j])>th):
            #if (eyu[0][i][j]!=0):
            #if (True):
                k0 = tf.cast(x[0][i][j]*255,tf.int16)
                for k in range(k0-1,k0+1+1,1):
                #for k in range(k0-ii,k0+ii+1,1):
                #for k in range(k0-ii,k0+ii+1,ii):
                    if ((k==k0) or (k<0) or (k>255)):
                        continue
                    #x1 = x*1
                    #x1[0][i][j] = k/255
                    xi1 = xi*1
                    xn1 = xi1.numpy()
                    xn1[0][i][j] = k
                    xi1 = tf.convert_to_tensor(xn1)
                    x1 = xi1/255
                    y1 = autoencoder.predict(x1[:1])
                    ex1 = x0-x1
                    ey1 = y0-y1
                    exu1 = tf.cast(ex1*255, tf.int16).numpy()
                    eyu1 = tf.cast(ey1*255, tf.int16).numpy()
                    #"""
                    cnt1 = 0
                    for m in range(0,28,1):
                        for n in range(0,28,1):
                            #if ((ey1[0][i][j]==ey[0][i][j]) or ((ey1[0][i][j]*ey[0][i][j]>0) and (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])))):
                            #if ((tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])) or (((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))))):
                            if (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])):
                            #if ((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))):                            
                                cnt1 = cnt1 +1
                    if (cnt1==(28*28)):
                    #if ((cnt1==(28*28)) and (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey)))):
                    #"""
                    #if ((k>k0) and (tf.reduce_mean(ey1)<0)):
                    #if ((((k<k0) and (tf.reduce_mean(ey1)>0)) or ((k>k0) and (tf.reduce_mean(ey1)<0))) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey)))):
                    #if (((k<k0) and (tf.reduce_mean(ey1)<tf.reduce_mean(ey))) or ((k>k0) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))))):
                    #if (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))):
                    #if ((tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))) and (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu)))):
                    #if (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))):
                    #if (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu))):
                        #x[0][i][j] = x1[0][i][j]
                        xi = xi1*1
                        x = xi/255
                        ey = ey1*1
                        eyu = eyu1*1
                        
                        xu = tf.cast(x*255, tf.uint8)       
                        print(ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())
                        file.write(str([ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
                        file.write('\n')
                
file.close()


0 0 0 0 45.400787 0.9864582
1 6 9 0 0 45.403748 0.9864582
1 7 4 0 0 45.406704 0.9864582
1 7 5 4 0 45.43341 0.9864582
1 7 6 83 84 45.44235 0.9864582
1 7 7 184 185 45.439365 0.9864582
1 7 8 159 159 45.44235 0.9864582
1 7 10 61 60 45.451305 0.98645824
1 7 12 3 0 45.47228 0.98645824
1 7 13 3 0 45.493355 0.9864583
1 7 14 1 0 45.490337 0.98645824
1 7 16 3 0 45.5115 0.98645824
1 7 17 2 0 45.52668 0.98645824
1 7 18 0 0 45.52972 0.98645824
1 7 19 1 0 45.538864 0.98645824
1 7 21 6 0 45.578693 0.98645824
1 8 3 0 0 45.58177 0.98645824
1 8 4 0 0 45.584854 0.98645824
1 8 6 221 222 45.58177 0.98645824
1 8 7 251 254 45.603386 0.98645824
1 8 9 252 254 45.61889 0.98645824
1 8 10 252 254 45.63445 0.98645824
1 8 11 241 241 45.63757 0.9864585
1 8 18 198 198 45.640694 0.98645836
1 8 19 198 198 45.643814 0.98645836
1 8 20 170 170 45.646942 0.98645836
1 8 21 51 52 45.643814 0.98645836
1 8 22 0 0 45.646942 0.98645836
1 9 4 0 0 45.65007 0.98645836
1 9 5 2 0 45.66574 0.98645836
1 9 8 72 72 45.668884 0.9864584
1 

*2-3 Step 3*

In [13]:
print(0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())

file = open("result_alr_1x1_searching_step_3.txt","a")
file.write(str([0,0,0,0,tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
file.write('\n')

#
#th = 0.001*tf.reduce_max(tf.abs(ey))
th = 0.005/255

#
for ii in range(1,20+1,1):
    for i in range(0,28,1):
        for j in range(0,28,1):
            """
            nb = 1
            for m in range(-1,+1):
                for n in range(-1,+1):
                    if (((i+m)>=0) and ((i+m)<28) and ((j+n)>=0) and ((j+n)<28) and (eyu[0][i+m][j+n]!=0)):
                        nb = nb+1
            if (nb>=1):
            """
            if (tf.abs(ey[0][i][j])>th):
            #if (eyu[0][i][j]!=0):
            #if (True):
                k0 = tf.cast(x[0][i][j]*255,tf.int16)
                for k in range(k0-1,k0+1+1,1):
                #for k in range(k0-ii,k0+ii+1,1):
                #for k in range(k0-ii,k0+ii+1,ii):
                    if ((k==k0) or (k<0) or (k>255)):
                        continue
                    #x1 = x*1
                    #x1[0][i][j] = k/255
                    xi1 = xi*1
                    xn1 = xi1.numpy()
                    xn1[0][i][j] = k
                    xi1 = tf.convert_to_tensor(xn1)
                    x1 = xi1/255
                    y1 = autoencoder.predict(x1[:1])
                    ex1 = x0-x1
                    ey1 = y0-y1
                    exu1 = tf.cast(ex1*255, tf.int16).numpy()
                    eyu1 = tf.cast(ey1*255, tf.int16).numpy()
                    """
                    cnt1 = 0
                    for m in range(0,28,1):
                        for n in range(0,28,1):
                            #if ((ey1[0][i][j]==ey[0][i][j]) or ((ey1[0][i][j]*ey[0][i][j]>0) and (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])))):
                            #if ((tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])) or (((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))))):
                            if (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])):
                            #if ((eyu1[0][i][j]==0) or (tf.abs(eyu1[0][i][j])<tf.abs(eyu[0][i][j]))):                            
                                cnt1 = cnt1 +1
                    if (cnt1==(28*28)):
                    #if ((cnt1==(28*28)) and (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey)))):
                    """
                    #if ((k>k0) and (tf.reduce_mean(ey1)<0)):
                    #if ((((k<k0) and (tf.reduce_mean(ey1)>0)) or ((k>k0) and (tf.reduce_mean(ey1)<0))) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey)))):
                    #if (((k<k0) and (tf.reduce_mean(ey1)<tf.reduce_mean(ey))) or ((k>k0) and (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))))):
                    #if (tf.abs(tf.reduce_mean(ey1))<tf.abs(tf.reduce_mean(ey))):
                    #if ((tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))) and (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu)))):
                    if (tf.abs(ey1[0][i][j])<tf.abs(ey[0][i][j])):
                    #if (tf.reduce_sum(tf.abs(ey1))<tf.reduce_sum(tf.abs(ey))):
                    #if (tf.reduce_sum(tf.abs(eyu1))<tf.reduce_sum(tf.abs(eyu))):
                        #x[0][i][j] = x1[0][i][j]
                        xi = xi1*1
                        x = xi/255
                        ey = ey1*1
                        eyu = eyu1*1
                        
                        xu = tf.cast(x*255, tf.uint8)       
                        print(ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy())
                        file.write(str([ii,i,j,k,xu0[0][i][j],tf.image.psnr(xu,xu0,255).numpy(),tf.image.ssim(tf.expand_dims(xu, axis=3),tf.expand_dims(xu0, axis=3),255,28)[0].numpy()]))
                        file.write('\n')
                
file.close()


0 0 0 0 70.08427 0.999941
1 11 4 0 0 71.05336 0.999941
1 12 9 0 0 72.30274 0.9999414
1 13 11 0 0 74.06366 0.99995893
1 13 13 0 0 77.07397 1.0
1 22 17 0 0 inf 1.0
